#1. Mounting Google Drive

This allows the notebook to import/export data from files inside Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#2. Installing ViZDoom

ViZDoom is the platform used to train the agents.
Here, the dependencies are installed, and then ViZDoom itself.

In [ ]:
%%bash
# Install deps from
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux
apt update
apt upgrade

apt install build-essential zlib1g-dev libsdl2-dev libjpeg-dev nasm tar libbz2-dev libgtk2.0-dev \
cmake git libfluidsynth-dev libgme-dev libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt install libboost-all-dev

# Lua binding dependencies
apt install liblua5.1-dev

apt update
apt upgrade

In [ ]:
!pip install git+https://github.com/mwydmuch/ViZDoom

#3. Importing libraries

Relevant libraries to the agent and its training are imported here.

In [ ]:
from vizdoom import *
#import vizdoom
import numpy as np
import pandas as pd

import time
import random

In [ ]:
#This ignores all warning messages that are normally displayed during training because of skimage
import warnings
warnings.filterwarnings('ignore')

#4. Create environment function

This function creates the environment and starts the game.
It returns an instance of the game and the list of possible actions.

In [ ]:
#Function that creates the environment, sets configuration, map and possible actions
#Input: Nothing
#Output: Game instance, list of possible actions
def create_environment():
    #New game instance
    game = DoomGame()

    #Adjust configuration file path
    #game.load_config("/home/werneck/Desktop/ViZDoom/agents/DQN/DQN_configuration.cfg")
    game.load_config('/content/drive/My Drive/ViZDoom/Config/random_configuration.cfg')

    #Adjust game scenario path
    #game.set_doom_scenario_path("/home/werneck/Desktop/ViZDoom/scenarios/basic.wad")
    game.set_doom_scenario_path('/content/drive/My Drive/ViZDoom/Maps/basic.wad')

    #Google Colab does not support the video output of ViZDoom. The following line is needed for the environment to be run.
    game.set_window_visible(False)

    #Game starts, no more configurations
    game.init()

    #Possible Actions
    #idle = [0, 0, 0, 1]
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]

    return game, possible_actions

#5. Create the data frame

The data frame will hold information regarding the experiments.

Data exported to files (like .csv files, for example) are stored in data frames.

In [ ]:
#DataFrame to hold training/testing data
df = pd.DataFrame(columns = ['Episode', 'Steps', 'Reward'])

#5. Instantiate the game and set some parameters

Instantiate the game.

Set number of episodes and the maximum steps per episode.
Render the game (even if offscreen).

In [ ]:
#Game creation call
game, possible_actions = create_environment()

In [ ]:
### TRAINING HYPERPARAMETERS
episodes = 50                   #Total episodes for training
max_steps = 100                 #Max possible steps in an episode

In [ ]:
# Turm this to False if you do not want to render the environment
episode_render = True

#6. General play loop

This starts the play loop for as many times as there are episodes.

Episode data is saved to a temporary data frame, and then appended to the main data frame.

In [ ]:
for episode in range(episodes):
    game.new_episode()
    step = 0
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        #misc = state.get_variables
        action = random.choice(possible_actions)
        reward = game.make_action(action)
        step += 1
        if (step == max_steps):
            break
    total_reward = game.get_total_reward()
    df_e = pd.DataFrame([[(episode+1), step, game.get_total_reward()]],
                        columns = ['Episode', 'Steps', 'Reward'])
    df = df.append(df_e, ignore_index = True)
    print ("Episode:", (episode+1), "Steps: ", step, "Result: ", total_reward)

#7. Ending the experiment

After all episodes are run, the instance of the game is closed.

A path is given for an output file, and then the information is exported from the data frame to the output file.

In [ ]:
game.close()
path = "/content/drive/My Drive/ViZDoom/Data/Random/test.csv"
df.to_csv(path, index = False)